# <center> <font style="color:rgb(100,109,254)">   Creating a Virtual Pen & Eraser </font> </center>


In [2]:
import cv2
import numpy as np
import time

In [3]:
cap, frame = None, None
# Intializing the webcam feed.
cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)

True

In [4]:
# This variable determines if we want to load color range from memory or use the ones defined here. 
load_from_disk = True

# If true then load color range from memory
if load_from_disk:
    penval = np.load('penval.npy')

# Creating A 5x5 kernel for morphological operations
kernel = np.ones((5,5),np.uint8)

## <font style="color:rgb(134,19,348)">Step 3: Tracking the Target Pen  </font>

In [5]:
def create_object_mask():
    global frame
    
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # If you're reading from memory then load the upper and lower ranges from there
    if load_from_disk:
            lower_range = penval[0]
            upper_range = penval[1]
            
    # Otherwise define your own custom values for upper and lower range.
    else:             
        lower_range  = np.array([26,80,147])
        upper_range = np.array([81,255,255])
    
    obj_mask = cv2.inRange(hsv, lower_range, upper_range)
    
    # Perform the morphological operations to get rid of the noise
    obj_mask = cv2.erode(obj_mask, kernel,iterations = 1)
    obj_mask = cv2.dilate(obj_mask, kernel,iterations = 2)
    
    return obj_mask

In [6]:
def draw_object_contour(obj_mask):
    
    # Find Contours in the frame.
    contours, hierarchy = cv2.findContours(obj_mask, cv2.RETR_EXTERNAL,
                                           cv2.CHAIN_APPROX_SIMPLE)
    
    # Make sure there is a contour present and also make sure its size is bigger than noise threshold.
    if contours and cv2.contourArea(max(contours, key = cv2.contourArea)) > noiseth:
        
        # Grab the biggest contour with respect to area
        c = max(contours, key = cv2.contourArea)
        
        # Get bounding box coordinates around that contour
        x,y,w,h = cv2.boundingRect(c)
        
        # Draw that bounding box
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,25,255),2)        

In [7]:
# This threshold is used to filter noise, the contour area must be bigger than this to qualify as 
# an actual contour.
noiseth = 500

In [8]:
while True:
    
    _, frame = cap.read()
    frame = cv2.flip( frame, 1 )

    obj_mask = create_object_mask()
    
    draw_object_contour(obj_mask)      

    cv2.imshow('image',frame)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break

cv2.destroyAllWindows()
cap.release()